# Imports

In [ ]:
import duckdb
import sklearn
import glob
import pandas as pd
import polars as pl
import plotly.express as px
import pgeocode
import hvplot as hv
import hvplot.pandas
import bokeh
import geopandas
import geodatasets
import json

In [ ]:
bokeh.__version__

# Code

In [ ]:
# g = geopandas.read_file('/home/jnovak/src/public/State-zip-code-GeoJSON/ca_california_zip_codes_geo.min.json')

In [ ]:
list(enumerate(glob.glob(
    '/home/jnovak/src/public/State-zip-code-GeoJSON/*.min.json'
)))

## Load zip shapes as GeoJSON

In [ ]:
def load_json_file(path):
    with open(path) as f:
        return json.load(f)

In [ ]:
zip_geojson = [
    load_json_file(f)
    for f in glob.glob(
        '/home/jnovak/src/public/State-zip-code-GeoJSON/*.min.json'
    )
]

In [ ]:
len(zip_geojson)

In [ ]:
zip_geojson[0].keys()

In [ ]:
zip_geojson[0]['type']

In [ ]:
sum([len(g['features']) for g in zip_geojson])

In [ ]:
len(zip_geojson[23]['features'])

In [ ]:
zip_geojson[23]['features'][0]

In [ ]:
json.con

## Load zip shapes into a GeoDataFrame

In [ ]:
zip_shapes = pd.concat(
    geopandas.read_file(f)
    for f in glob.glob(
    '/home/jnovak/src/public/State-zip-code-GeoJSON/*.min.json'
))

In [ ]:
zip_shapes.info()

In [ ]:
zip_shapes

In [ ]:
print(geodatasets.data.keys())

In [ ]:
print('\n'.join(geodatasets.data.flatten().keys()))

In [ ]:
duckdb.sql('''
SELECT COUNT(*) AS c
FROM '/mnt/c/Users/me/Downloads/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv'
''')

## Load Zillow price history

In [ ]:
zillow = duckdb.read_csv('/mnt/c/Users/me/Downloads/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')

In [ ]:
zillow.describe()

In [ ]:
zillow

In [ ]:
duckdb.sql('SELECT COUNT(DISTINCT RegionId) FROM zillow')

In [ ]:
duckdb.sql('''SELECT * FROM zillow WHERE RegionName = '90292' LIMIT 5''')

In [ ]:
duckdb.sql('''SELECT * FROM zillow WHERE City = 'Los Angeles' ORDER BY RegionID LIMIT 5''')

In [ ]:
nomi = pgeocode.Nominatim('us')

In [ ]:
type(nomi.query_postal_code('79997'))

In [ ]:
locations = nomi.query_postal_code(zillow.to_df().RegionName.astype(str).to_list())

In [ ]:
locations

In [ ]:
locations.describe()

In [ ]:
locations.info()

In [ ]:
locations.postal_code.iloc[0]

In [ ]:
locations.set_index('postal_code').loc['90292']

In [ ]:
locations.set_index('postal_code').loc['90048']

In [ ]:
len(locations)

## Join Zillow stats to Zip centroids

In [ ]:
zillow_locs = duckdb.sql('''
SELECT *
FROM zillow
LEFT JOIN locations ON (
  zillow.RegionName = locations.postal_code
)
''')

## Join Zillow stats to Zip polygons

In [ ]:
zillow_zip_shapes = zillow.to_df().join(
    zip_shapes.set_index('ZCTA5CE10'),
    on='RegionName',
    how='left'
)

In [ ]:
zillow_zip_shapes[[
    'RegionID',
    'RegionName',
    'State',
    'City',
    'Metro',
    'CountyName',
    '2024-10-31',
    'geometry',
]].info(max_cols=1000)

In [ ]:
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 0)

In [ ]:
zillow_zip_shapes.head(1).T

In [ ]:
zillow_locs

# Plots

In [ ]:
zillow_locs.to_df().head().plot.scatter('longitude', 'latitude')

## Plot points

In [ ]:
hv.hvPlot(zillow_locs.to_df()).points(
    'longitude',
    'latitude',
    tiles=True,
    color='2024-10-31',
    cmap='coolwarm',
    # cmap='viridis',
    # color='red',
    alpha=0.5
)

## Plot geometry

In [ ]:
hv.hvPlot(zillow_zip_shapes).polygons(
    geo=True,
    tiles=True,
    # c='2024-10-31'
    # hover_cols='all'
)

## Prepare cleaned Zillow dataset

In [ ]:
zillow_zip_clean = geopandas.GeoDataFrame(zillow_zip_shapes[[
    # 'RegionID',
    'RegionName',
    'State',
    'City',
    'Metro',
    'CountyName',
    '2024-10-31',
    'geometry',
]].dropna().set_index('RegionName'))

In [ ]:
zillow_zip_clean.info()

In [ ]:
zillow_zip_clean.head()

In [ ]:
zillow_zip_clean.query(
    'State == "CA"'
)

In [ ]:
type(zillow_zip_clean)

In [ ]:
type(geopandas.GeoDataFrame(zillow_zip_clean))

In [ ]:
zillow_zip_clean['2024-10-31'].isna().value_counts()

In [ ]:
zillow_zip_clean.geometry.isna().value_counts()

## Plot with hvPlot + Bokeh

In [ ]:
hv.hvPlot(zillow_zip_clean.query(
    'CountyName == "Los Angeles County"'
)).polygons(
    geo=True,
    tiles=True,
    alpha=0.25,
    color='2024-10-31',
    cmap='YlGnBu',
    hover_cols='all',
    frame_width=500
)

In [ ]:
zillow_zip_clean.query(
    'CountyName == "Los Angeles County"'
).head()

## Plot chloropleth with Plotly Express

In [ ]:
df = duckdb.sql('''
SELECT RegionName, State, City, Metro, CountyName, "2024-10-31"
FROM zillow
-- LIMIT 10
''').to_df()
df

In [ ]:
import sys
sys.getsizeof(zip_geojson)

In [ ]:
sys.getsizeof(df)

In [ ]:
df.info(memory_usage='deep')

In [ ]:
px.choropleth_map(
    df,
    locations='RegionName',
    geojson=zip_geojson[23],
    featureidkey='properties.ZCTA5CE10',
    zoom=3,
    center={"lat": 37.0902, "lon": -95.7129},
    # geojson=zip_shapes,
    color='2024-10-31',
    opacity=0.5
    # scope='usa'
)

In [ ]:
geopandas.GeoDataFrame(zillow_zip_clean[['2024-10-31', 'geometry']]).plot('2024-10-31', legend=True)